In [1]:
import pandas as pd
import numpy as np

In [2]:
epsilon = 1e-6

In [75]:
from google.colab import files
uploaded = files.upload()

Saving ConsumerDefence_sector_predictions.csv to ConsumerDefence_sector_predictions.csv


In [76]:
# Load sector CSV (Technology sector in this example)
df = pd.read_csv("ConsumerDefence_sector_predictions.csv")

In [77]:
# Use epsilon instead of dropping rows
df["Volatility"] = df["Volatility"].replace(0, epsilon)
df["Log_Returns"] = df["Log_Returns"].replace([np.inf, -np.inf], np.nan).fillna(0)
df["Volatility"] = df["Volatility"].replace([np.inf, -np.inf], np.nan).fillna(epsilon)

In [78]:
# Dictionary to hold scores
results = {
    "Sharpe": {},
    "Kelly": {},
    "Sortino": {},
    "Custom": {}
}

In [79]:
# Compute heuristics
for stock in df["Stock"].unique():
    stock_df = df[df["Stock"] == stock]
    r = stock_df["Log_Returns"]
    v = stock_df["Volatility"].replace(0, epsilon)
    downside = r[r < 0]
    downside_std = downside.std() if downside.std() != 0 else epsilon

    sharpe_scores = r / v
    kelly_scores = (r - r.var()) / v
    sortino_scores = r / downside_std
    custom_scores = sharpe_scores * sortino_scores

    results["Sharpe"][stock] = np.median(sharpe_scores)
    results["Kelly"][stock] = np.median(kelly_scores)
    results["Sortino"][stock] = np.median(sortino_scores)
    results["Custom"][stock] = np.median(custom_scores)


In [80]:
# Normalize each equation's scores
normalized_results = {}
for eqn in results:
    scores = pd.Series(results[eqn])
    min_score = scores.min()
    max_score = scores.max()
    normalized = (scores - min_score) / (max_score - min_score + epsilon)
    normalized_results[eqn] = normalized

In [ ]:
# Print equation blocks
for eqn_name, score_series in normalized_results.items():
    print(f"\n== {eqn_name} Equation Block ==")
    for stock, score in score_series.items():
        print(f"{stock} - heuristic: {round(score, 4)} - equation name: {eqn_name}")

In [82]:
# Determine best heuristic and equation per stock
final_output = []
for stock in df["Stock"].unique():
    best_eqn = max(normalized_results, key=lambda eq: normalized_results[eq][stock])
    best_score = normalized_results[best_eqn][stock]
    sector = df[df["Stock"] == stock]["Sector"].iloc[0]
    final_output.append({
        "Stock": stock,
        "Sector": sector,
        "Best Heuristic": round(best_score, 4),
        "Best Equation": best_eqn
    })


In [ ]:
final_df = pd.DataFrame(final_output)
print("\n== Final Best Heuristics per Stock ==")
print(final_df)

In [84]:
#Save to CSV if needed
final_df.to_csv("ConsumerDefence_stocks_heuristics.csv", index=False)